In [ ]:
from multiprocessing import Pool
from nemesis.event_generation.generate_multiCPU import events_wrapped
import pickle


In [ ]:
first_seed = 4500001
event_types = ['cascades', 'tracks', 'starting_tracks']
starting_it = 10


for it in range(10):
    indexes_all = []
    seed_start = first_seed + 1600*it
    event_type = event_types[0]
    v_x = (1 + it)/100
    print(v_x)
    #v_x = 0
    n_proc = 15
    n_events_per_process = 100
    seed_starts = range(seed_start, seed_start+n_proc*n_events_per_process, n_events_per_process)
    for v in seed_starts:
        indexes_all.append([event_type, v_x, v])
    
    if __name__ == '__main__':
        with Pool(n_proc) as p:
            all_res = p.map(events_wrapped, indexes_all)
    
    events = []
    records = []
    for i in range(0, len(all_res)):
        for j in range(100):
            events.append(all_res[i][0][j])
            records.append(all_res[i][1][j])
    pickle.dump((events, records), open(f"/dss/pone/pone_events/{event_type}{len(events)}_vx{v_x}.pickle", "wb"))
    print(f'iteration number {it} completed!')

In [ ]:
first_seed = 2600001
event_types = ['cascades', 'tracks', 'starting_tracks']
starting_it = 10

for it in range(10):
    indexes_all = []
    seed_start = first_seed + 1600*it
    event_type = event_types[1]
    v_x = (0.5 + it)/100
    #v_x = 0
    n_proc = 15
    n_events_per_process = 100
    seed_starts = range(seed_start, seed_start+n_proc*n_events_per_process, n_events_per_process)
    for v in seed_starts:
        indexes_all.append([event_type, v_x, v])
    
    if __name__ == '__main__':
        with Pool(n_proc) as p:
            all_res = p.map(events_wrapped, indexes_all)
    
    events = []
    records = []
    for i in range(0, len(all_res)):
        for j in range(100):
            events.append(all_res[i][0][j])
            records.append(all_res[i][1][j])
    pickle.dump((events, records), open(f"/dss/pone/pone_events/{event_type}{len(events)}_vx{v_x}.pickle", "wb"))
    print(f'iteration number {it} completed!')

In [ ]:
first_seed = 2700002
event_types = ['cascades', 'tracks', 'starting_tracks']
starting_it = 10

for it in range(10):
    indexes_all = []
    seed_start = first_seed + 1600*it
    event_type = event_types[2]
    v_x = (0.5 + it)/100
    #v_x = 0
    n_proc = 15
    n_events_per_process = 100
    seed_starts = range(seed_start, seed_start+n_proc*n_events_per_process, n_events_per_process)
    for v in seed_starts:
        indexes_all.append([event_type, v_x, v])
    
    if __name__ == '__main__':
        with Pool(n_proc) as p:
            all_res = p.map(events_wrapped, indexes_all)
    
    events = []
    records = []
    for i in range(0, len(all_res)):
        for j in range(100):
            events.append(all_res[i][0][j])
            records.append(all_res[i][1][j])
    pickle.dump((events, records), open(f"/dss/pone/pone_events/{event_type}{len(events)}_vx{v_x}.pickle", "wb"))
    print(f'iteration number {it} completed!')

In [1]:
import pickle
import os
import functools
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="0.8"
import sys
sys.path.insert(0, '../olympus')
sys.path.insert(0, '../hyperion')
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from itertools import product
import awkward as ak
import pandas as pd
from olympus.event_generation.photon_propagation.norm_flow_photons import make_generate_norm_flow_photons, make_nflow_photon_likelihood
from olympus.event_generation.photon_propagation.utils import sources_to_model_input
from nemesis.event_generation.detector import Detector, make_line, make_triang
from olympus.event_generation.event_generation import (
    generate_cascade,
    generate_cascades,
    simulate_noise,
    generate_realistic_track,
    generate_realistic_tracks,
    generate_realistic_tracks_test,
    generate_realistic_starting_tracks,)
from olympus.event_generation.lightyield import make_pointlike_cascade_source, make_realistic_cascade_source
from olympus.event_generation.utils import sph_to_cart_jnp, proposal_setup

#from olympus.plotting import plot_event
from hyperion.medium import medium_collections
from hyperion.constants import Constants
import jax
jax.config.update('jax_platform_name', 'cpu')
from jax import random
from jax import numpy as jnp
import json

path_to_config = "../hyperion/data/pone_config_optimistic.json"
config = json.load(open(path_to_config))["photon_propagation"]
ref_ix_f, sca_a_f, sca_l_f, _ = medium_collections[config["medium"]]

def c_medium_f(wl):
    """Speed of light in medium for wl (nm)."""
    return Constants.BaseConstants.c_vac / ref_ix_f(wl)

rng = np.random.RandomState(31338)
oms_per_line = 20
dist_z = 50 # m
dark_noise_rate = 16 * 1e-5  # 1/ns
side_len = 100 # m
pmts_per_module = 16
pmt_cath_area_r = 75E-3 / 2 # m
module_radius = 0.21 # m



/home/work/.local/lib/python3.10/site-packages/jax/_src/api_util.py:222: SyntaxWarning: Jitted function has invalid argnames {'interactions'} in static_argnames. Function does not take these args.This warning will be replaced by an error after 2022-08-20 at the earliest.
  warnings.warn(f"Jitted function has invalid argnames {invalid_argnames} "


In [2]:
from multiprocessing import Pool
import pickle
import numpy as np
import os

def load_test_events(v_x):
    path_to_events = "/dss/pone/pone_events"
    det, cascades, cascade_records = pickle.load(open(os.path.join(path_to_events, f"cascades1500_vx{v_x}.pickle"), "rb"))
    det, tracks, track_records = pickle.load(open(os.path.join(path_to_events, f"tracks1500_vx{v_x}.pickle"), "rb"))
    det, stracks, strack_records = pickle.load(open(os.path.join(path_to_events, f"starting_tracks1500_vx{v_x}.pickle"), "rb"))
    
    all_ev = [det, cascades, cascade_records, tracks, track_records, stracks, strack_records]
    
    return all_ev
    
    
cascades_test, cascade_records_test = [], []
tracks_test, track_records_test = [], []
stracks_test, strack_records_test = [], []
det_test = []
v_x_all = []

for it in range(1,21):
    v_x_all.append(np.round(it*0.005, 3))

for i in range(2):
    v_x = v_x_all[(y*10):((y+1)*10)]
    if __name__ == '__main__':
        with Pool(10) as p:
            all_ev = p.map(load_test_events, v_x)
    print(f"iteration {i} done")


    for v in range(len(all_ev)):
        det_test.append(all_ev[v][0])
        cascades_test += all_ev[v][1]
        cascade_records_test += all_ev[v][2]
        tracks_test += all_ev[v][3]
        track_records_test += all_ev[v][4]    
        stracks_test += all_ev[v][5]
        strack_records_test += all_ev[v][6]

iteration 0 done


KeyboardInterrupt: 